# Instructlab local - 01 Prepare environment

## 1. Install dependencies

In [6]:
!nvidia-smi --query-gpu=name,memory.total --format csv

name, memory.total [MiB]
NVIDIA GeForce RTX 4090, 24564 MiB


In [11]:
import subprocess
result = subprocess.run("nvidia-smi | grep 'CUDA Version'", shell=True, capture_output=True, text=True)
result.stdout

'| NVIDIA-SMI 550.76.01              Driver Version: 552.44         CUDA Version: 12.4     |\n'

In [ ]:
pip install torch==2.3.0

In [3]:
from importlib.metadata import version
version('torch')

'2.3.0'

In [ ]:
pip install transformers==4.41.1

In [6]:
version('transformers')

'4.41.1'

In [ ]:
pip install accelerate==0.30.1

In [7]:
version('accelerate')

'0.30.1'

In [ ]:
pip install flash-attn==2.5.8 --no-build-isolation

In [10]:
version('flash_attn')

'2.5.8'

In [ ]:
pip install vllm==0.4.3

In [4]:
version('vllm')

'0.4.3'

In [ ]:
pip install gradio==4.31.5

In [10]:
version('gradio')

'4.31.5'

*Specific dependency for Phi3 small*

In [ ]:
pip install pytest==8.2.1

In [51]:
version('pytest')

'8.2.1'

*Specific dependency for Mixtral 8x7B HQQ quantization*

In [ ]:
pip install hqq==0.1.7.post2

In [58]:
version('hqq')

'0.1.7.post2'

WARNING: need to patch a bug in HQQ v0.1.7.post2

> vi /workspace/instructlab-local/.venv/lib/python3.10/site-packages/hqq/models/hf/base.py

BaseHQQHFModel.create_model()

```python
    # Replace this line
    def create_model(cls, save_dir, kwargs):
        
    # With this line
    def create_model(cls, save_dir, **kwargs):    
```

## 2. Download student and teacher models

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

import os
import torch
from transformers.utils.hub import cached_file

memory_unit_mb = 1024*1024
memory_unit_gb = 1024*1024*1024

def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

def get_model_path_and_size_on_disk(pretrained_model_id):    
    model_config_file = cached_file(pretrained_model_id, "config.json", local_files_only=True)
    model_directory = os.path.dirname(os.path.dirname(model_config_file))    
    total_size = get_directory_size(model_directory)
    return model_directory,total_size

def get_gpu_free_memory():
    return (torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0))/memory_unit_mb

def display_model_properties(model_name):
    path,size = get_model_path_and_size_on_disk(model_name)
    print(f"- model files size   : {(size/memory_unit_gb):.2f} GB")
    print(f"- stored in directory: {path}")

    free_mem_mb = get_gpu_free_memory()
    print(f"- free memory after load: {free_mem_mb:.2f} MB")

Note: if you need to clean the huggingface cache before downloading new models ...

```bash
cd /workspace/instructlab-local/
source .venv/bin/activate

pip install huggingface_hub[cli]

huggingface-cli delete-cache --dir /models/huggingface/transformers
```

Then navigate among model revisions with the arrow key, press space to select the revisions to delete, then Enter to delete.

### 2.1 Mistral 7B base and instruct v0.3

In [33]:
model_name = "mistralai/Mistral-7B-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map=0, torch_dtype="auto", attn_implementation="flash_attention_2")

display_model_properties(model_name)
del model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

- model files size   : 13.50 GB
- stored in directory: /models/huggingface/transformers/models--mistralai--Mistral-7B-v0.3/snapshots
- free memory after load: 10226.98 MB


In [34]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map=0, torch_dtype="auto", attn_implementation="flash_attention_2")

display_model_properties(model_name)
del model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

- model files size   : 13.50 GB
- stored in directory: /models/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.3/snapshots
- free memory after load: 10226.98 MB


### 2.2 Llama 3 8B base and instruct

In [38]:
model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map=0, torch_dtype="auto", attn_implementation="flash_attention_2")

display_model_properties(model_name)
del model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

- model files size   : 14.97 GB
- stored in directory: /models/huggingface/transformers/models--meta-llama--Meta-Llama-3-8B/snapshots
- free memory after load: 9246.98 MB


In [39]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map=0, torch_dtype="auto", attn_implementation="flash_attention_2")

display_model_properties(model_name)
del model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

- model files size   : 14.97 GB
- stored in directory: /models/huggingface/transformers/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots
- free memory after load: 9246.98 MB


### 2.3 Phi 3 mini and small instruct

In [42]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map=0, torch_dtype="auto", attn_implementation="flash_attention_2")

display_model_properties(model_name)
del model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

- model files size   : 7.12 GB
- stored in directory: /models/huggingface/transformers/models--microsoft--Phi-3-mini-4k-instruct/snapshots
- free memory after load: 17275.10 MB


In [55]:
model_name = "microsoft/Phi-3-small-8k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, use_safetensors=True, device_map=0, torch_dtype="auto", attn_implementation="flash_attention_2", trust_remote_code=True)

display_model_properties(model_name)
del model

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-small-8k-instruct:
- tokenization_phi3_small.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-small-8k-instruct:
- configuration_phi3_small.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-small-8k-instruct:
- modeling_phi3_small.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

- model files size   : 13.77 GB
- stored in directory: /models/huggingface/transformers/models--microsoft--Phi-3-small-8k-instruct/snapshots
- free memory after load: 10335.75 MB


### 2.4 Mixtral 8x7B instruct v0.1 HQQ - 3.5 bits and 2 bits

In [2]:
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer as HQQAutoTokenizer

/workspace/instructlab-local/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
model_name = "mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-3bit-metaoffload-HQQ"

tokenizer = HQQAutoTokenizer.from_pretrained(model_name)
model = HQQModelForCausalLM.from_quantized(model_name)

free_mem_mb = get_gpu_free_memory()
print(f"- free memory after load: {free_mem_mb:.2f} MB")
    
del model

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 32/32 [00:08<00:00,  3.66it/s]


- free memory after load: 4049.73 MB


In [ ]:
model_name = "mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-2bit_g16_s128-HQQ"

tokenizer = HQQAutoTokenizer.from_pretrained(model_name)
model = HQQModelForCausalLM.from_quantized(model_name)

free_mem_mb = get_gpu_free_memory()
print(f"- free memory after load: {free_mem_mb:.2f} MB")

del model

In [5]:
free_mem_mb = get_gpu_free_memory()
print(f"- free memory after load: {free_mem_mb:.2f} MB")

- free memory after load: 6853.60 MB
